### Import Necessary Libraries

In [71]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging
)

from peft import (
    LoraConfig,
    PeftModel,
    prepare_model_for_kbit_training,
    get_peft_model
)

import os, torch, wandb
from datasets import load_dataset
from trl import SFTTrainer, setup_chat_format
from dotenv import load_dotenv
import bitsandbytes as bnb

### Huggingface and WandB authentication

In [62]:
load_dotenv()

HF_KEY = os.getenv("HUGGINGFACEHUB_API_TOKEN")

In [64]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HF_KEY

In [7]:
os.environ["WANDB_NOTEBOOK_NAME"] = "C:/Users/User/Data Science/Deep Learning/Generative AI/Fine Tuning LLMs/fine-tuning llama 3.2 1B/research.ipynb"

In [ ]:
wandb.login()

In [5]:
run = wandb.init(
    project='Fine-tune Llama 3.2 on Customer Support Dataset',
    job_type="training",
    anonymous="allow"
)

In [8]:
base_model = "meta-llama/Llama-3.2-1B-Instruct"
new_model = "llama-3.2-3b-it-Ecommerce-ChatBot"
dataset_name = "bitext/Bitext-customer-support-llm-chatbot-training-dataset"

### Set the data type and attention implementation

In [9]:
if torch.cuda.get_device_capability()[0] >= 8:
    !pip install -qqq flash-attn
    torch_dtype = torch.bfloat16
    attn_implementation = "flash_attention_2"
else:
    torch_dtype = torch.float16
    attn_implementation = "eager"

### QLoRA Config

In [10]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch_dtype,
    bnb_4bit_use_double_quant=True
)

### Load Model

In [13]:
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map="auto",
    attn_implementation=attn_implementation
)

model.safetensors:  60%|######    | 1.49G/2.47G [00:00<?, ?B/s]

c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\accelerate\utils\modeling.py:856: UserWarning: expandable_segments not supported on this platform (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\c10/cuda/CUDAAllocatorConfig.h:28.)
  _ = torch.tensor([0], device=i)


generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

### Load Tokenizer

In [14]:
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

### Importing the dataset

In [86]:
dataset = load_dataset(dataset_name, split="train")

FIrst, we will train with only 1000 samples of the dataset

In [87]:
test_dataset = dataset.shuffle(seed=65).select(range(1000, 1200))

In [88]:
len(test_dataset)

200

In [36]:
len(dataset['instruction'])

1000

### Data Preprocessing to match our chat template

In [44]:
instruction = """You are a top-rated customer service agent named John. 
    Be polite to customers and answer all their questions.
    """

In [51]:
def format_chat_template(row):
    from transformers import AutoTokenizer

    base_model = "meta-llama/Llama-3.2-1B-Instruct"
    tokenizer = AutoTokenizer.from_pretrained(
        base_model, trust_remote_code=True)

    instruction = """You are a top-rated customer service agent named John. 
        Be polite to customers and answer all their questions.
        """

    row_json = [{"role": "system", "content": instruction},
                {"role": "user", "content": row["instruction"]},
                {"role": "assistant", "content": row["response"]}]

    row["text"] = tokenizer.apply_chat_template(row_json, tokenize=False)
    return row

In [52]:
dataset = dataset.map(format_chat_template, num_proc=2)

Map (num_proc=2):   0%|          | 0/1000 [00:00<?, ? examples/s]

In [89]:
test_dataset = test_dataset.map(format_chat_template, num_proc=2)

Map (num_proc=2):   0%|          | 0/200 [00:00<?, ? examples/s]

In [69]:
dataset.push_to_hub("customer-support-1k", token=HF_KEY)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/Victorano/customer-support-1k/commit/157c1da2fb0e842461161ea69d8551fc26715b4d', commit_message='Upload dataset', commit_description='', oid='157c1da2fb0e842461161ea69d8551fc26715b4d', pr_url=None, pr_revision=None, pr_num=None)

In [90]:
test_dataset.push_to_hub("customer-support-test-200", token=HF_KEY)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/Victorano/customer-support-test-200/commit/af6cd0525f81ae93586b3035fb3e5adb819a651b', commit_message='Upload dataset', commit_description='', oid='af6cd0525f81ae93586b3035fb3e5adb819a651b', pr_url=None, pr_revision=None, pr_num=None)

In [70]:
dataset['text'][3]

"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 Jul 2024\n\nYou are a top-rated customer service agent named John. \n        Be polite to customers and answer all their questions.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\ncould you tell me about the options for shipping?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nOf course, I'd be delighted to provide you with information about our shipping options! Here are the various choices we offer:\n\n1. Standard Shipping: This option typically arrives within {{Date Range}} business days, catering to non-urgent items and ensuring a cost-effective delivery.\n\n2. Expedited Shipping: If you're looking for a faster option, choose expedited shipping. Your items will reach you within {{Date Range}} business days, offering a balance between speed and affordability.\n\n3. Overnight Shipping: For urgent needs, we have overnight shipping. This ensures your item

### Setting Up the Model
extract the linear model name from the model

In [77]:
def find_all_linear_names(model):
    cls = bnb.nn.Linear4bit
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, cls):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])
    if 'lm_head' in lora_module_names:  # needed for 16 bit
        lora_module_names.remove('lm_head')
    return list(lora_module_names)

In [78]:
modules = find_all_linear_names(model)

In [79]:
modules

['q_proj', 'o_proj', 'v_proj', 'down_proj', 'gate_proj', 'k_proj', 'up_proj']

Using the Linear Module name to create the LoRA Adapter, this is only what we will be fine tuning

### Lora Config

In [80]:
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=modules
)

In [81]:
model, tokenizer = setup_chat_format(model, tokenizer)
model = get_peft_model(model, peft_config)

### SFT Hyperparameter

In [82]:
training_arguments = TrainingArguments(
    output_dir=new_model,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=2,
    optim="paged_adamw_32bit",
    num_train_epochs=1,
    eval_strategy="steps",
    eval_steps=0.2,
    logging_steps=1,
    warmup_steps=10,
    logging_strategy="steps",
    learning_rate=2e-4,
    fp16=False,
    bf16=False,
    group_by_length=True,
    report_to="wandb"
)

### Supervised Fine Tuning

In [83]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    peft_config=peft_config,
    max_seq_length=512,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments,
    packing=False
)

KeyError: "Column train not in the dataset. Current columns in the dataset: ['flags', 'instruction', 'category', 'intent', 'response', 'text']"